# Preliminary Data Exploration and Mapping of OCO2 Data
#### NOTE: If you have maps generated in this notebook, it will run very slowly. I advise you clear the outputs when done

In [ ]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import urllib.request
import json


import matplotlib.pyplot as plt
import plotly.express as px
import plotly

import geopandas as gpd

#BASEMAP
import mpl_toolkits
import seaborn as sns; sns.set()
import datetime as dt


## Read in a CSV created in NASA_data_gather.ipynb

In [ ]:
#This file is read from a local source because it was too large to upload to github
df_xco2_base= pd.read_csv(r"C:\Users\ddrye\OneDrive\Documents\OMSA_Program\OMSA 2023\Summer2023\Practicum\off_git\data\OCO2_BASE_2014-2023_V1.csv")

## Overview for 2016, 2017 and 2018 data

In [ ]:

sedac_years=df_xco2_base[df_xco2_base['Year'].isin([2016,2017,2018])]

print("non filtered shape:",sedac_years.shape)

#filtering on good quality data
sedac_years_filtered=sedac_years[sedac_years["xco2_quality_flag"]==0]

#dropping the unnamed column - just a redundant index column.
sedac_years_filtered=sedac_years_filtered.drop(["Unnamed: 0"],axis=1)

#getting the filtered shape of the data
print("filtered shape:",sedac_years_filtered.shape)

#getting a look at the data
display(sedac_years_filtered.head())
display(sedac_years_filtered.tail())

#checking for NA values
if sedac_years_filtered[sedac_years_filtered.isna().any(axis=1)].empty==True:
    print("No NAs")
else:
    print("NAs Present")
    display(sedac_years_filtered[sedac_years_filtered.isna().any(axis=1)])

#getting an overview of the xco2 readings
print("\n xco2 readings:")
display(sedac_years_filtered.xco2.describe())

#getting the average co2 reading accross all years
xco2_max=sedac_years_filtered['xco2'].max()
print(xco2_max)
print(sedac_years_filtered.query('xco2 == xco2.max()'))

## Overview for all of the years

In [ ]:
#df_xco2_base= pd.read_csv(r"C:\Users\ddrye\OneDrive\Documents\OMSA_Program\OMSA 2023\Summer2023\Practicum\off_git\data\OCO2_BASE_2014-2023_V1.csv")
print("non filtered shape:",df_xco2_base.shape)

#filtering on good quality data
df_xco2_base_filtered=df_xco2_base[df_xco2_base["xco2_quality_flag"]==0]

#dropping the unnamed column - just a redundant index column and the total_delta column - poor naming on my part 
df_xco2_base_filtered=df_xco2_base_filtered.drop(["Unnamed: 0"],axis=1)

#getting the filtered shape of the data
print("filtered shape:",df_xco2_base_filtered.shape)

#getting a look at the data
display(df_xco2_base_filtered.head())
display(df_xco2_base_filtered.tail())

#checking for NA values
if df_xco2_base_filtered[df_xco2_base_filtered.isna().any(axis=1)].empty==True:
    print("No NAs")
else:
    print("NAs Present")
    display(df_xco2_base_filtered[df_xco2_base_filtered.isna().any(axis=1)])

#getting an overview of the xco2 readings
print("\n xco2 readings:")
display(df_xco2_base_filtered.xco2.describe())

#getting the average co2 reading accross all years
xco2_max=df_xco2_base_filtered['xco2'].max()
print(xco2_max)
print(df_xco2_base_filtered.query('xco2 == xco2.max()'))



## Groupings for Visualization - FOR ALL YEARS

In [ ]:
#A function that makes grouping by a little easier
def overview(df,groupby=[]):
    counts=df.groupby(groupby, as_index=False)["xco2"].size().rename(columns={'size':'readings_count'})
    xco2_avg=df.groupby(groupby, as_index=False)["xco2"].mean().rename(columns={'xco2':'avg_xco2'})
    xco2_min=df.groupby(groupby, as_index=False)["xco2"].max().rename(columns={'xco2':'max_xco2'})
    xco2_max=df.groupby(groupby, as_index=False)["xco2"].min().rename(columns={'xco2':'min_xco2'})
    std_deviation=df.groupby(groupby, as_index=False)["xco2"].std().rename(columns={'xco2':'stddev_xco2'})

    combined=pd.merge(counts, xco2_avg, on=groupby)
    combined=pd.merge(combined, std_deviation, on=groupby)
    combined=pd.merge(combined, xco2_min, on=groupby)
    combined=pd.merge(combined, xco2_max, on=groupby)

    if 'geoid' in groupby:
        combined = combined.astype({'geoid':'string'})
        combined["geoid"] = combined["geoid"].str.zfill(5)
    
    return combined


#Group by state and year
state_ov=overview(df_xco2_base_filtered,groupby=["state_name","Year"])
display(state_ov.head())

#Group by county_name and geoid - some countys may have the same name, thats why we're doing geoid as well
county_ov=overview(df_xco2_base_filtered,groupby=["county_name","geoid"])
display(county_ov.head())


## Plotting


In [ ]:
#Group by year and month
month_year_ov=overview(df_xco2_base_filtered,groupby=["Year",'Month'])

#setting the year range for 2015 through 2022
month_year_ov = month_year_ov.loc[(month_year_ov['Year'] < 2023) & (month_year_ov['Year'] > 2014) ]

#creating a datetime column from month and year
month_year_ov['Date']=month_year_ov['Month'].map(str) + '-' + month_year_ov['Year'].map(str)
month_year_ov['Date'] = pd.to_datetime(month_year_ov['Date']).dt.strftime('%m-%Y')
#display(month_year_ov.head())

#plotting
a= sns.lineplot(x='Date', y='avg_xco2', data=month_year_ov)
a.set(title='Average XCO2 Over Time')
a.set_ylabel("Column Averaged CO2 (ppm)",fontsize=15)
plt.xlabel('Date', fontsize=10)
plt.ylabel('Column Averaged CO2 (ppm)', fontsize=10)
plt.tick_params(axis='x', which='major', labelsize=8, rotation=90)
plt.tick_params(axis='y', which='major', labelsize=8)
for index, label in enumerate(a.get_xticklabels()):
   if index % 4 == 0:
      label.set_visible(True)
   else:
      label.set_visible(False)

In [ ]:
#Seasonality - USING ALL YEARS
seasonal_ov=overview(df_xco2_base_filtered,groupby=["Month"])

#plotting
b= sns.lineplot(x='Month', y='avg_xco2', data=seasonal_ov)
b.set(title='Avg XCO2 Seasonality (2015-2022)')
#a.set_xlabel("Month",fontsize=15)
b.set_ylabel("Column Averaged CO2 (ppm)",fontsize=15)
plt.xlabel('Month', fontsize=10)
plt.ylabel('Column Averaged CO2 (ppm)', fontsize=10)
plt.tick_params(axis='x', which='major', labelsize=8)
plt.tick_params(axis='y', which='major', labelsize=8)


# Mapping

#### Getting the county polygons for visualization on map

In [ ]:
request = urllib.request.Request('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
opener = urllib.request.build_opener()
response = opener.open(request)
counties_map = json.load(response)

In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

#state_ov=overview(sedac_years_filtered,groupby=["state_name","Year"])

#creating a datetime column from month and year
sedac_years_filtered['Date']=sedac_years_filtered['Month'].map(str) + '-' + sedac_years_filtered['Year'].map(str)
sedac_years_filtered['Date'] = pd.to_datetime(sedac_years_filtered['Date']).dt.strftime('%m-%Y')
sedac_years_filtered_2016=sedac_years_filtered[sedac_years_filtered['Year']==2016]

display(sedac_years_filtered_2016.head())


In [ ]:
fig = px.scatter_mapbox(sedac_years_filtered_2016, lat="Latitude", lon="Longitude", width=800,
                           height=600)
fig.show()

In [ ]:
df_xco2_base_filtered_years=df_xco2_base_filtered[df_xco2_base_filtered['Year'].isin([2015,2016,2017,2018,2019,2020,2021,2022])]
ysc_ov=overview(df_xco2_base_filtered_years,groupby=['geoid',"county_name","Year"])
ysc_ov['Date'] = pd.to_datetime(ysc_ov['Year'].map(str)).dt.strftime('%Y')
ysc_ov.sort_values(by='Date', inplace=True)
ysc_ov.rename(columns={'avg_xco2':'XCO2 (ppm)'}, inplace=True)


fig = px.choropleth(ysc_ov, geojson=counties_map, locations='geoid', color='readings_count',
                           color_continuous_scale="Purples",
                           animation_frame='Date',
                           hover_data=['readings_count','county_name'],
                           scope="usa",
                           range_color=(0,100),
                           width=800,
                           height=600,
                           )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},legend_title="CO2 ppm")
fig.show()

#### Mapping County Average XCO2 from year to year

In [ ]:
#Year State and County for all years
df_xco2_base_filtered_years=df_xco2_base_filtered[df_xco2_base_filtered['Year'].isin([2015,2016,2017,2018,2019,2020,2021,2022])]
ysc_ov=overview(df_xco2_base_filtered_years,groupby=['geoid',"county_name","Year"])
ysc_ov['Date'] = pd.to_datetime(ysc_ov['Year'].map(str)).dt.strftime('%Y')
ysc_ov.sort_values(by='Date', inplace=True)
ysc_ov.rename(columns={'avg_xco2':'XCO2 (ppm)'}, inplace=True)


fig = px.choropleth(ysc_ov, geojson=counties_map, locations='geoid', color='XCO2 (ppm)',
                           color_continuous_scale="Viridis",
                           animation_frame='Date',
                           hover_data=['readings_count','county_name'],
                           scope="usa",
                           range_color=(390,420),
                           width=800,
                           height=600,
                           )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},legend_title="CO2 ppm")
fig.show()

#### Mapping Average XCO2 over all Years

In [ ]:
#County for 2015-2018
df_xco2_base_filtered_years=df_xco2_base_filtered[df_xco2_base_filtered['Year'].isin([2016,2017,2018])]
c_ov=overview(df_xco2_base_filtered_years,groupby=['geoid',"county_name"])

#plotting
fig = px.choropleth(c_ov, geojson=counties_map, locations='geoid', color='avg_xco2',
                           color_continuous_scale="Viridis",
                           hover_data=['readings_count','county_name'],
                           scope="usa",
                           range_color=(400,405),
                           width=800,
                           height=600
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#### Mapping percent change from year to year for all years

In [ ]:
#County percent change from year to year
df_xco2_base_filtered_allyears=df_xco2_base_filtered[df_xco2_base_filtered['Year'].isin([2015,2016,2017,2018,2019,2020,2021,2022])]
cypct_ov=overview(df_xco2_base_filtered_allyears,groupby=['geoid',"county_name",'Year'])
pct_change = (cypct_ov.groupby('geoid')['avg_xco2']
                                  .apply(pd.Series.pct_change) + 1).rename('pct_change').reset_index()
cypct_ov=pd.merge(cypct_ov, pct_change['pct_change'], left_index=True, right_index=True)
cypct_ov['Date'] = pd.to_datetime(cypct_ov['Year'].map(str)).dt.strftime('%Y')


fig = px.choropleth(cypct_ov, geojson=counties_map, locations='geoid', color='pct_change',
                           color_continuous_scale="Viridis",
                           hover_data=['readings_count','county_name'],
                           scope="usa",
                           animation_frame='Date',
                           range_color=(1.00,1.025),
                           width=800,
                           height=600
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#### Mapping percent increase from 2016 to 2018

In [ ]:
#County percent change from 2016 to 2018
df_xco2_base_filtered_years=df_xco2_base_filtered[df_xco2_base_filtered['Year'].isin([2016,2018])]
cpct_ov=overview(df_xco2_base_filtered_years,groupby=['geoid',"county_name",'Year'])
pct_change = (cpct_ov.groupby('geoid')['avg_xco2']
                                  .apply(pd.Series.pct_change) + 1).rename('pct_change').reset_index()
cpct_ov=pd.merge(cpct_ov, pct_change['pct_change'], left_index=True, right_index=True)
cpct_ov=cpct_ov[cpct_ov['Year']==2018]

#plotting
fig = px.choropleth(cpct_ov, geojson=counties_map, locations='geoid', color='pct_change',
                           color_continuous_scale="YlOrRd",
                           hover_data=['readings_count','county_name'],
                           scope="usa",
                           range_color=(1.0,1.025),
                           width=800,
                           height=600
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()